In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch

C:\Users\vitek\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# === Config ===
DATA_PATH_IN  = "preprocessed_data.csv"
DATA_PATH_OUT = "finbert_sentiment.csv" 

ID_COL        = "id"
TEXT_COL      = "sentiment_ready_text"

# === Load data ===
df = pd.read_csv(DATA_PATH_IN)

print("Input shape:", df.shape)
df.head()


Input shape: (13663, 13)


,id,sentiment_ready_text,type,subreddit,created_utc,normalized_score,mentioned_tickers,n_tickers,text_length,word_count,date,hour,day_of_week
0,1hqr72t,good roe return equity quick data analysis s s...,post,investing,2024-12-31 23:56:46,0.019045,TOP,1,1072,171,2024-12-31,23,1
1,1hqqmq2,soundhound soun market cap 75x s annualized re...,post,stocks,2024-12-31 23:24:29,0.039443,SOUN,1,476,82,2024-12-31,23,1
2,1hqqgv5,smci beginning play p e looking low current va...,post,stocks,2024-12-31 23:15:10,0.011074,SMCI,1,419,75,2024-12-31,23,1
3,1hqpxkb,thoughts rddt feedback appreciated,post,investing,2024-12-31 22:45:36,0.018076,RDDT,1,42,6,2024-12-31,22,1
4,1hqpw3r,d r horton dhi good value stock 7 ytd forward ...,post,ValueInvesting,2024-12-31 22:43:25,0.051121,DHI,1,391,75,2024-12-31,22,1


In [3]:
# === Load FinBERT model ==
MODEL_NAME = "ProsusAI/finbert"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

device = 0 if torch.cuda.is_available() else -1
print("Using device:", "GPU" if device == 0 else "CPU")

# === Build sentiment pipeline ===
sentiment_pipe = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    device=device,
    return_all_scores=True,
    truncation=True,
    max_length=128            # can increase to 256 if your texts are longer
)


Device set to use cpu


Using device: CPU


C:\Users\vitek\AppData\Roaming\Python\Python313\site-packages\transformers\pipelines\text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [4]:
def score_texts(texts):
    """
    Run FinBERT on a list of texts and return structured sentiment info.

    Parameters
    ----------
    texts : list of str
        The input texts to classify.

    Returns
    -------
    results : list of dict
        Each dict has:
        - sentiment_label : str       ('positive', 'neutral', 'negative')
        - sentiment_score : float     (p_pos - p_neg in [-1, 1])
        - p_pos, p_neu, p_neg : float (probabilities)
    """
    # This calls the HF pipeline once for the whole batch
    outputs = sentiment_pipe(texts)

    results = []
    for out in outputs:
        # out is a list like:
        # [{'label': 'positive', 'score': 0.7}, {'label': 'neutral', 'score': 0.2}, {'label': 'negative', 'score': 0.1}]
        # Normalize label names to lowercase to be robust to variations
        probs = {d["label"].lower(): float(d["score"]) for d in out}

        p_pos = probs.get("positive", 0.0)
        p_neg = probs.get("negative", 0.0)
        p_neu = probs.get("neutral", 0.0)

        # Continuous sentiment score in [-1, 1]
        sentiment_score = p_pos - p_neg

        # Discrete label = argmax over the three probabilities
        sentiment_label = max(probs, key=probs.get)

        results.append({
            "sentiment_label": sentiment_label,
            "sentiment_score": sentiment_score,
            "p_pos": p_pos,
            "p_neu": p_neu,
            "p_neg": p_neg
        })

    return results


In [5]:
# === Batch configuration ===
# batch size kdyztak budeme menit 
BATCH_SIZE = 32

sentiment_labels = []
sentiment_scores = []
p_pos_list = []
p_neu_list = []
p_neg_list = []

# Replace NA with empty strings so the model doesn't crash
texts = df[TEXT_COL].fillna("").tolist()

n_texts = len(texts)
print("Number of texts to process:", n_texts)

for start in range(0, n_texts, BATCH_SIZE):
    end = start + BATCH_SIZE
    batch = texts[start:end]

    scored = score_texts(batch)

    # Extend our result lists
    for r in scored:
        sentiment_labels.append(r["sentiment_label"])
        sentiment_scores.append(r["sentiment_score"])
        p_pos_list.append(r["p_pos"])
        p_neu_list.append(r["p_neu"])
        p_neg_list.append(r["p_neg"])

    # progress print
    if (start // BATCH_SIZE) % 50 == 0:
        print(f"Processed {min(end, n_texts)} / {n_texts} texts")

print("Scores computed:", len(sentiment_labels), "rows in df:", len(df))


Number of texts to process: 13663
Processed 32 / 13663 texts
Processed 1632 / 13663 texts
Processed 3232 / 13663 texts
Processed 4832 / 13663 texts
Processed 6432 / 13663 texts
Processed 8032 / 13663 texts
Processed 9632 / 13663 texts
Processed 11232 / 13663 texts
Processed 12832 / 13663 texts
Scores computed: 13663 rows in df: 13663


In [9]:
# Optional: save to disk
df.to_csv(DATA_PATH_OUT, index=False)
print("Saved FinBERT sentiment data to:", DATA_PATH_OUT)


Saved FinBERT sentiment data to: finbert_sentiment.csv
